
## Importante: para rodar a rotina é necessário ter o CDS API instalado

*PASSOS PARA INSTALAR O CDS API*

>**1. Cadastro**

Fazer o cadastro no site da Climate Data Store https://cds.climate.copernicus.eu/#!/home

>**2. Adicionar a chave e instalando o pacote CDS API no Python**

No link https://cds.climate.copernicus.eu/api-how-to#install-the-cds-api-key mostra como adicionar a chave e como realizar a instalação do biblioteca CDS API no Python.

A instalação do pacote pode ser feito pelo gerenciador de pacotes PIP ou CONDA\
*ex: pip install cdsapi\
ou\
conda install cdsapi*

***Caso encontrem dificuldades em instalar o CDS API, o vídeo no link a seguir mostra o passo a passo detalhadamente.***
https://www.youtube.com/watch?v=NHbvfggMC10&ab_channel=CopernicusECMWF

<hr style="border:1px solid lightblue"> </hr>

* *A rotina estar configurada para realizar aquisição dos dados horários (a cada 1h) do ERA5, para todos os dias e meses do intervalo de tempo desejado.* 

* *A única alteração que deve ser feita pelo usuário é determinar os diretórios para os arquivos de saída.*

* *Os parâmetros **latitude, longitude e intervalo de tempo** serão pedidos pela própria rotina quando executada.*  


In [ ]:
import xarray as xr
from cdo import Cdo
import pandas as pd
import numpy
import cdsapi
import os
import glob

cdo = Cdo()
c = cdsapi.Client()

In [ ]:
value1=input("Digite somente o primeiro ano da série temporal(mínimo 1979):\n")
value2=input("Digite somente o último ano da série temporal(máximo 2021):\n")
cor1=input("Digite o limite Norte da região (90 até -90; apenas número inteiro):\n")
cor3=input("Digite o limite Sul da região (90 até -90; apenas número inteiro):\n")
cor2=input("Digite o limite Oeste da região (-180 até 180; apenas número inteiro):\n")
cor4=input("Digite o limite Leste da região (-180 até 180; apenas número inteiro):\n")

value1=int(value1)
value2=int(value2)+1
cor1=int(cor1)
cor2=int(cor2)
cor3=int(cor3)
cor4=int(cor4)

In [ ]:

for year in range(value1,value2):
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': 'total_precipitation',
        'year': str(year),
        'month':['0'+str(x) if x<10 else str(x) for x in range(1,2)
        ], 
        'day': ['0'+str(x) if x<10 else str(x) for x in range(1,32)
        ],
        'time':['0'+str(x)+':00' if x<10 else str(x)+':00' for x in range(24)],
        'area': [
            cor1, cor2, cor3,
            cor4
        ],
    },
    '/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA_raw/era5_year'+str(year)+'.nc')

### Atenção!
Antes de execurtar os comandos da janela seguinte, deixe um diretório exclusivo para os outputs da função CDO.DAYMEAN. Isso será necessário pois a função OPEN_MFDATASET vai agrupar todos os arquivos .nc desse diretório.

In [ ]:
for year2 in range(value1, value2):
    infile = '/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA_raw/era5_year' + str(year2) + '.nc'
    cdo.daymean(input=infile, 
                output='/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA5_mean/' + str(year2) + '.nc')
    
dado = xr.open_mfdataset('/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA5_mean/*.nc')
dado = dado.drop_dims("bnds")
dado.load().to_netcdf('/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA5_concat/Precipitation.nc') 

In [ ]:
# files = glob.glob('/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA_raw/*.nc', recursive=True)
# files2 = glob.glob('/home/lipfp/Documentos/Marine_Heatwaves/saidas/ERA5_mean/*.nc', recursive=True)
# for f in files:
#         os.remove(f)
# for f in files2:
#         os.remove(f)